In [46]:
import sys
sys.path.append("../")

import pandas as pd
import plotly.graph_objects as go
from dateutil import parser
import datetime as dt

from api.oanda_api import OandaApi
from instrumentCollection.instrument_collection import instrumentCollection as ic
from DB.db import DataDB

pd.set_option('display.max_rows', None)

In [47]:
%pip install statsmodels

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
database = DataDB()

In [49]:
ic.LoadInstrumentsDB()

In [50]:
pairs = []
for pair, val in ic.instruments_dict.items():
    if val.ins_type == "CURRENCY" and "GBP" in pair:
        pairs.append(pair)

In [51]:
pairs

['GBP_SGD',
 'GBP_USD',
 'GBP_ZAR',
 'GBP_HKD',
 'GBP_CAD',
 'GBP_CHF',
 'GBP_PLN',
 'GBP_NZD',
 'EUR_GBP',
 'GBP_JPY',
 'GBP_AUD']

In [52]:
data = []
api = OandaApi()
for p in pairs:
    d_temp = api.get_candles_df(pair_name=p, granularity="D",count=400)
    d_temp['gain'] = ((d_temp.mid_c - d_temp.mid_o) / d_temp.mid_c) * 100
    d_temp['pair'] = p
    if '_GBP' in p:
        d_temp['gain'] = d_temp['gain'] * -1
    data.append(d_temp)
candles_df = pd.concat(data)
candles_df.reset_index(drop=True, inplace=True)
candles_df.time = candles_df.time.dt.date

In [53]:
candles_df.tail()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,gain,pair
4384,2023-04-13,152362,1.84673,1.85452,1.83981,1.85039,1.84511,1.85444,1.83955,1.84976,1.84835,1.85461,1.84005,1.85102,0.197796,GBP_AUD
4385,2023-04-16,135417,1.85084,1.85488,1.84580,1.84692,1.84834,1.85478,1.84570,1.84680,1.85334,1.85498,1.84589,1.84703,-0.212245,GBP_AUD
4386,2023-04-17,141946,1.84631,1.85183,1.84004,1.84677,1.84381,1.84997,1.83990,1.84667,1.84881,1.85433,1.84017,1.84687,0.024908,GBP_AUD
4387,2023-04-18,125637,1.84844,1.85668,1.84410,1.85280,1.84594,1.85658,1.84398,1.85269,1.85094,1.85677,1.84418,1.85291,0.235320,GBP_AUD
4388,2023-04-19,133312,1.85129,1.85524,1.84040,1.84547,1.84879,1.85517,1.84032,1.84535,1.85379,1.85549,1.84048,1.84559,-0.315367,GBP_AUD


In [54]:
calendar_data = database.query_all(DataDB.CALENDAR_COLL)

In [55]:
calendar_data_df = pd.DataFrame.from_dict(calendar_data)

In [56]:
calendar_data_df.date = calendar_data_df.date.dt.date

In [57]:
calendar_data_df_uk = calendar_data_df[calendar_data_df.country == "united kingdom"].copy()

In [58]:
calendar_data_df_uk.head(10)

,date,country,category,event,symbol,actual,previous,forecast
6,2023-04-13,united kingdom,monthly gdp mom,gdp mom,GBRMGM,0%,0.4%,0.1%
13,2023-04-18,united kingdom,unemployment rate,unemployment rate,UKUEILOR,3.8%,3.7%,3.8%
14,2023-04-18,united kingdom,claimant count change,claimant count change,UNITEDKINCLACOUCHA,28.2K,-18.8K,-9.5K
18,2023-04-19,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,10.1%,10.4%,10.2%
20,2023-04-20,united kingdom,consumer confidence,gfk consumer confidence,UKCCI,-30,-36,-34
22,2023-04-21,united kingdom,retail sales mom,retail sales mom,GBRRetailSalesMoM,,1.2%,-0.5%


In [59]:
calendar_data_df_uk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 6 to 22
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      6 non-null      object
 1   country   6 non-null      object
 2   category  6 non-null      object
 3   event     6 non-null      object
 4   symbol    6 non-null      object
 5   actual    6 non-null      object
 6   previous  6 non-null      object
 7   forecast  6 non-null      object
dtypes: object(8)
memory usage: 432.0+ bytes


In [60]:
for col in ['actual', 'previous', 'forecast']:
    for sy in ['£', '%', 'B', 'K']:
        calendar_data_df_uk[col] = calendar_data_df_uk[col].str.replace(sy, "")
    calendar_data_df_uk[col].replace('', 0, inplace=True)
    calendar_data_df_uk[col] = calendar_data_df_uk[col].astype(float)

In [61]:
date_match = {}
for orig in calendar_data_df_uk.date.unique():
    d = orig
    tries = 0
    while d not in candles_df.time.values:
        d = d + dt.timedelta(days=1)
        tries += 1
        if tries > 5:
            print("Failed")
            break
    date_match[orig] = d

Failed
Failed


In [62]:
date_match

{datetime.date(2023, 4, 13): datetime.date(2023, 4, 13),
 datetime.date(2023, 4, 18): datetime.date(2023, 4, 18),
 datetime.date(2023, 4, 19): datetime.date(2023, 4, 19),
 datetime.date(2023, 4, 20): datetime.date(2023, 4, 26),
 datetime.date(2023, 4, 21): datetime.date(2023, 4, 27)}

In [63]:
calendar_data_df_uk['orig_date'] = calendar_data_df_uk.date
calendar_data_df_uk.date = [date_match[x] for x in calendar_data_df_uk.date]

In [64]:
calendar_data_df_uk['delta_prev'] = calendar_data_df_uk.actual - calendar_data_df_uk.previous
calendar_data_df_uk['delta_fc'] = calendar_data_df_uk.actual - calendar_data_df_uk.forecast

In [65]:
calendar_data_df_uk.head(4)

,date,country,category,event,symbol,actual,previous,forecast,orig_date,delta_prev,delta_fc
6,2023-04-13,united kingdom,monthly gdp mom,gdp mom,GBRMGM,0.0,0.4,0.1,2023-04-13,-0.4,-0.1
13,2023-04-18,united kingdom,unemployment rate,unemployment rate,UKUEILOR,3.8,3.7,3.8,2023-04-18,0.1,0.0
14,2023-04-18,united kingdom,claimant count change,claimant count change,UNITEDKINCLACOUCHA,28.2,-18.8,-9.5,2023-04-18,47.0,37.7
18,2023-04-19,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,10.1,10.4,10.2,2023-04-19,-0.3,-0.1


In [66]:
candles_an = candles_df[['time', 'pair', 'gain']].copy()

In [67]:
candles_an.head(2)

,time,pair,gain
0,2021-10-07,GBP_SGD,-0.276026
1,2021-10-10,GBP_SGD,-0.176442


In [68]:
merged = pd.merge(left=candles_an, right=calendar_data_df_uk, left_on='time', right_on='date')

In [69]:
merged[merged.category=='inflation rate'].head(5)

,time,pair,gain,date,country,category,event,symbol,actual,previous,forecast,orig_date,delta_prev,delta_fc
33,2023-04-19,GBP_SGD,-0.081430,2023-04-19,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,10.1,10.4,10.2,2023-04-19,-0.3,-0.1
34,2023-04-19,GBP_USD,0.010447,2023-04-19,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,10.1,10.4,10.2,2023-04-19,-0.3,-0.1
35,2023-04-19,GBP_ZAR,-0.624084,2023-04-19,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,10.1,10.4,10.2,2023-04-19,-0.3,-0.1
36,2023-04-19,GBP_HKD,0.019451,2023-04-19,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,10.1,10.4,10.2,2023-04-19,-0.3,-0.1
37,2023-04-19,GBP_CAD,0.132983,2023-04-19,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,10.1,10.4,10.2,2023-04-19,-0.3,-0.1


In [70]:
merged.category.unique()

array(['monthly gdp mom', 'unemployment rate', 'claimant count change',
       'inflation rate'], dtype=object)

In [76]:
c = 'inflation rate'
df_an = merged[merged.category==c]
print(df_an[df_an.delta_fc >= 0].gain.sum())
print(df_an[df_an.delta_fc < 0].gain.sum())

0.0
-2.035585262708778


In [79]:
import plotly.express as px

In [87]:
cat = 'inflation rate'
df_cat = merged[merged.category==cat]
for p in pairs:
    df_plot = df_cat[df_cat.pair == p]
    print(p)
    fig = px.scatter(df_plot, x="gain", y="delta_prev", trendline="ols")
    fig.show()

GBP_SGD


GBP_USD


GBP_ZAR


GBP_HKD


GBP_CAD


...

GBP_NZD


EUR_GBP


GBP_JPY


GBP_AUD


Empty DataFrame
Columns: []
Index: []
